In [1]:
import numpy as np
import datetime
import os
import pandas as pd

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
todaystring = datetime.date.today().strftime("%y%m%d")

import ModelFitFunctions
from ModelFitFunctions import *

# Set Params

In [2]:
filepath = 'data'

wbefile = 'data/wastewater.csv'

prevfile = 'data/clinical.csv'
namestring = '_qpcr_fits_'
earlynamestring = '_qpcr_fits_trainearly_'

catches = ['Stickney','Calumet','OBrien']
prevenddate = {'positive tests':'2022-11-01', 
               'test positivity rate':'2022-04-06', 
               'hospital admissions':'2022-07-15', 
               'Cook County beds in use':'2022-09-20'}
        
fitwwtps = ['comb']
allprev = ['positive tests', 'test positivity rate', 'hospital admissions', 'Cook County beds in use']
trymodels = ['raw_data','only_flow_norm','pmmov_norm','only_bcov_norm','bcov_flow_norm','pmmov_bcov_norm','pmmov_bcov_flow_norm']
whatiscomb = catches
myend = '2022-01-01'
altend = '2022-01-01'
lags = np.arange(-10,11)
ww_lod = 67

In [3]:
wbe = pd.read_csv(wbefile, parse_dates=['date'])
wbe.rename(columns={'wrp':'catchment','sarscov2':'raw_data'},inplace=True)

prevdf = pd.read_csv(prevfile, parse_dates=['date'])
prevdf.rename(columns={'wrp':'catchment'},inplace=True)

wbe = Fix_wbe(wbe)


In [4]:
prevlagdict,prevbounds = get_prevlags(prevdf,catches,allprev,prevenddate,'2020-08-01','2022-04-01',corthresh=0.00129,wlags=[-2,5],plotcors=False)


Correlating: positive tests test positivity rate
	max lag = 3 	other lags =  [3 4]
	 [0, 10]
 Possible offsets: [[3], [4]]
 wlags = [[-2, 5], [0, 10]]

Correlating: positive tests hospital admissions
	max lag = 2 	other lags =  [2 3]
	 [-1, 9]
Correlating: test positivity rate hospital admissions
	max lag = 1 	other lags =  [0 1]
	 [-1, 9]
 Possible offsets: [[3, 3]]
 wlags = [[-2, 5], [0, 10], [-1, 9]]

Correlating: positive tests Cook County beds in use
	max lag = 9 	other lags =  [ 8  9 10]
	 [5, 16]
Correlating: test positivity rate Cook County beds in use
	max lag = 8 	other lags =  [7 8]
	 [5, 16]
Correlating: hospital admissions Cook County beds in use
	max lag = 7 	other lags =  [6 7]
	 [5, 16]
 Possible offsets: [[3, 3, 10]]
 wlags = [[-2, 5], [0, 10], [-1, 9], [5, 16]]

{'positive tests': 0, 'test positivity rate': 3, 'hospital admissions': 3, 'Cook County beds in use': 10}


,prevind,lower,upper
0,positive tests,-2,5
1,test positivity rate,-3,7
2,hospital admissions,-4,6
3,Cook County beds in use,-5,6


In [5]:
paramdf,aicdf,pdf,aicfulldf = FitModelParameters(wbe,prevdf,fitwwtps,allprev,trymodels,whatiscomb,myend,lags,ww_lod,prevlagdict,prevenddate)

lag: -10 , catch:  comb
lag: -9 , catch:  comb
lag: -8 , catch:  comb
lag: -7 , catch:  comb
lag: -6 , catch:  comb
lag: -5 , catch:  comb
lag: -4 , catch:  comb
lag: -3 , catch:  comb
lag: -2 , catch:  comb
lag: -1 , catch:  comb
lag: 0 , catch:  comb
lag: 1 , catch:  comb
lag: 2 , catch:  comb
lag: 3 , catch:  comb
lag: 4 , catch:  comb
lag: 5 , catch:  comb
lag: 6 , catch:  comb
lag: 7 , catch:  comb
lag: 8 , catch:  comb
lag: 9 , catch:  comb
lag: 10 , catch:  comb


In [6]:
paramdf.to_csv(os.path.join(filepath,'fitparams'+namestring+todaystring+'.csv'),index=False)
aicdf.to_csv(os.path.join(filepath,'fitaic'+namestring+todaystring+'.csv'),index=False)

In [7]:
print(namestring+todaystring)

_qpcr_fits_230825


In [8]:
paramdf,aicdf,pdf,aicfulldf = FitModelParameters(wbe,prevdf,fitwwtps,allprev,trymodels,whatiscomb,'2021-08-01',lags,ww_lod,prevlagdict,prevenddate,altend=altend)

lag: -10 , catch:  comb
lag: -9 , catch:  comb
lag: -8 , catch:  comb
lag: -7 , catch:  comb
lag: -6 , catch:  comb
lag: -5 , catch:  comb
lag: -4 , catch:  comb
lag: -3 , catch:  comb
lag: -2 , catch:  comb
lag: -1 , catch:  comb
lag: 0 , catch:  comb
lag: 1 , catch:  comb
lag: 2 , catch:  comb
lag: 3 , catch:  comb
lag: 4 , catch:  comb
lag: 5 , catch:  comb
lag: 6 , catch:  comb
lag: 7 , catch:  comb
lag: 8 , catch:  comb
lag: 9 , catch:  comb
lag: 10 , catch:  comb


In [9]:
paramdf.to_csv(os.path.join(filepath,'fitparams'+earlynamestring+todaystring+'.csv'),index=False)
aicdf.to_csv(os.path.join(filepath,'fitaic'+earlynamestring+todaystring+'.csv'),index=False)
aicfulldf.to_csv(os.path.join(filepath,'fitfullaic'+earlynamestring+todaystring+'.csv'),index=False)

In [10]:
print(earlynamestring+todaystring)

_qpcr_fits_trainearly_230825
